In [1]:
%%capture
!pip install wandb
!pip install modin[all]
!pip install ray --upgrade
# !pip install wrapt_timeout_decorator

In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: a-sh0ts. Use `wandb login --relogin` to force relogin


True

In [3]:
project_name = "krea-open-prompts"
run_name = "process-open-prompts-sd"
tags = ["download", "stable_diffusion", "process"]
_config = {
    # option of "prompts" or "sample_prompts"
    "dataset": "sample_prompts"
}

In [4]:
run = wandb.init(project=project_name, name=run_name, tags=tags, config=_config)

In [5]:
config = run.config
dataset_name = config['dataset']

In [6]:
art = run.use_artifact('open-prompts-sd:latest', type='raw_data')
dataset_path = art.get_path(f"{dataset_name}.csv").download()

In [7]:
import ray
ray.init(ignore_reinit_error=True, dashboard_host="0.0.0.0", include_dashboard=True)

2022-09-27 13:57:09,944	INFO worker.py:1515 -- Started a local Ray instance. View the dashboard at http://10.138.0.33:8265 


Python version:,3.7.12
Ray version:,2.0.0
Dashboard:,http://10.138.0.33:8265


In [8]:
import pandas as pd
import modin.pandas as mpd
# from modin.config import ProgressBar
# ProgressBar.enable()

In [9]:
raw_df = pd.read_csv(dataset_path)

In [10]:
import json

In [11]:
from pandas import json_normalize

In [12]:
raw_data = json_normalize(raw_df["raw_data"].apply(json.loads).tolist(), sep="_")

In [13]:
df = raw_df[["prompt"]].join([raw_data])

In [14]:
#BUG: check if modin actually helps here
# df = mpd.DataFrame(df)

In [15]:
df = df.drop(["modifiers"], axis=1)

In [16]:
from PIL import Image
import requests

In [17]:
from pathlib import Path
import os

In [18]:
# dataset_name = "test_modin"

In [19]:
image_folder = Path(".", dataset_name)
image_folder.mkdir(parents=True, exist_ok=True)

In [20]:
default_timeout = 5

In [21]:
def download_image_files(image_url):
    file_name = image_url.rsplit('/', 1)[-1]
    file_path = Path(image_folder, file_name)
    try:
        #Allows for retries without redownloads
        if file_path.exists():
            #Test the image actually opens and then close it
            img = Image.open(file_path)
            img.close()
            return (str(file_path), True)
        #Downloads image and writes it to file
        img_data = requests.get(image_url, timeout=default_timeout).content
        with open(file_path, 'wb') as handler:
            handler.write(img_data)
        #Test the image actually opens and then close it
        img = Image.open(file_path)
        img.close()
        return (str(file_path), True)
    except Exception as e:
        print(e)
        #Remove traces of erred files to prevent broken files from still existing
        if file_path.exists():
            os.remove(file_path)
        return (None, False)

In [22]:
from tqdm import tqdm
tqdm.pandas()

In [23]:
responses = df["raw_discord_data_image_uri"].progress_apply(download_image_files)

100%|██████████| 998/998 [00:00<00:00, 2871.06it/s]

cannot identify image file '/home/jupyter/krea-marqo/sample_prompts/beautiful_coy_natural_redhead_cottagecore_peasant_maiden_intricate_elegant_highly_detailed_digital_painting_artstation_concept_art_smooth_sh_-H_768_-n_9_-i_-S_216508618_ts-1659853734_idx-6.png'


In [24]:
df["local_image_location"] = [response[0] for response in responses]
df["image_download_success"] = [response[1] for response in responses]

In [25]:
df.shape

(998, 17)

In [26]:
processed_df = df[df["image_download_success"]]

In [27]:
processed_df.shape

(997, 17)

In [28]:
processed_df.columns

Index(['prompt', 'image_uri', 'thumbnail_uri', 'raw_discord_data_seed',
       'raw_discord_data_width', 'raw_discord_data_height',
       'raw_discord_data_content', 'raw_discord_data_is_grid',
       'raw_discord_data_num_step', 'raw_discord_data_cfg_scale',
       'raw_discord_data_image_uri', 'raw_discord_data_timestamp',
       'raw_discord_data_content_type', 'raw_discord_data_image_proxy_uri',
       'raw_discord_data_num_generations', 'local_image_location',
       'image_download_success'],
      dtype='object')

In [29]:
processed_df.iloc[0]

prompt                              A portrait photo of a kangaroo wearing an oran...
image_uri                                                                     PENDING
thumbnail_uri                                                                 PENDING
raw_discord_data_seed                                                     556046175.0
raw_discord_data_width                                                            512
raw_discord_data_height                                                           512
raw_discord_data_content                                                             
raw_discord_data_is_grid                                                          0.0
raw_discord_data_num_step                                                        50.0
raw_discord_data_cfg_scale                                                       15.0
raw_discord_data_image_uri          https://cdn.discordapp.com/attachments/1005543...
raw_discord_data_timestamp                            

In [30]:
processed_df["local_image_location"].iloc[0]

'sample_prompts/A_portrait_photo_of_a_kangaroo_wearing_an_orange_hoodie_and_blue_sunglasses_standing_on_the_grass_in_front_of_the_Sydney_Opera_House_holding_a_sign_-C_15.0_-n_9_-i_-S_556046175_ts-1660001285_idx-4.png'

In [31]:
selected_columns = [
    "prompt",
    "local_image_location",
    "raw_discord_data_seed",
    "raw_discord_data_width",
    "raw_discord_data_height",
    "raw_discord_data_is_grid",
    "raw_discord_data_num_step",
    "raw_discord_data_cfg_scale",
    "raw_discord_data_timestamp",
    "raw_discord_data_num_generations"
    
]

In [32]:
processed_df = processed_df[selected_columns]

In [33]:
processed_df.iloc[0]

prompt                              A portrait photo of a kangaroo wearing an oran...
local_image_location                sample_prompts/A_portrait_photo_of_a_kangaroo_...
raw_discord_data_seed                                                     556046175.0
raw_discord_data_width                                                            512
raw_discord_data_height                                                           512
raw_discord_data_is_grid                                                          0.0
raw_discord_data_num_step                                                        50.0
raw_discord_data_cfg_scale                                                       15.0
raw_discord_data_timestamp                                               1660001285.0
raw_discord_data_num_generations                                                  9.0
Name: 0, dtype: object

In [34]:
processed_df_path = f"{dataset_name}.csv"
processed_df.to_csv(processed_df_path, index=False)

In [35]:
processed_data_art = wandb.Artifact(name=dataset_name, type="processed_data")

In [36]:
processed_data_art.add_file(processed_df_path)
processed_data_art.add_dir(image_folder, name=dataset_name)

wandb: Adding directory to artifact (./sample_prompts)... Done. 0.9s


In [37]:
run.log_artifact(processed_data_art)

In [38]:
run.finish()